In [ ]:
import re
import pandas as pd
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.options.plotting.backend = "plotly"

In [ ]:
cc = pd.read_csv('data/443/DVT443_clamps.csv', usecols=[0,4,10])
cc['name'] = [re.sub(r'\s+', '', name) for name in cc['name']] # remove whitespace
cc.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)
cc['type'] = ['OrientationClamp' if 'OC' in name else 'CentralizerClamp' if 'CC' in name else 'Other' for name in cc['name']]
cc['hadware_name'] = [name.split('.')[0] for name in cc['name']]
cc['obs'] = [name.split('-')[1] if '-' in name else pd.NA for name in cc['name']]

#cc = cc.loc[cc['type'] == 'OrientationClamp']


# perfs = pd.read_csv('data\mabee2606_perfs_BIASED.csv', usecols=[0,1,7])
# perfs['type'] = 'PERFS'
# perfs.rename(columns={'perfAngleFromHighSideClockwiseDegrees': 'AngleFromHighSideClockwiseDegrees'}, inplace=True)

In [ ]:
features = cc[['type', 'hadware_name', 'depth', 'AngleFromHighSideClockwiseDegrees']].groupby(['type', 'hadware_name']).median(numeric_only=True)
features = features.sort_values(by=['depth'], ascending=False).reset_index()
features['fiberAngleFromHighSideClockwiseDegrees'] = [angle+180 if angle < 180 else angle-180 for angle in features['AngleFromHighSideClockwiseDegrees']]
features['fiber_plot_angle'] = [angle-360 if angle > 180 else angle for angle in features['fiberAngleFromHighSideClockwiseDegrees']]


# combined = pd.concat([features, perfs])
combined = features
combined['plot_angle'] = [angle-360 if angle > 180 else angle for angle in combined['AngleFromHighSideClockwiseDegrees']]
combined['angle_rounded'] = combined['AngleFromHighSideClockwiseDegrees'].round(0)
combined['fiber_angle_rounded'] = combined['fiberAngleFromHighSideClockwiseDegrees'].round(0)

In [ ]:
fig = go.Figure()

# Add windows
gozone_svg = ''.join([f'M {xy[0][0]+30},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+30},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='OrientationClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='OrientationClamp', ['plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-30},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-30},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='OrientationClamp', ['plot_angle','depth']].values,range(combined.loc[combined['type']=='OrientationClamp', ['plot_angle','depth']].shape[0]))][::-1])

nogozone_svg = ''.join([f'M {xy[0][0]+10},{xy[0][1]} ' if xy[1]==0 else f'L{xy[0][0]+10},{xy[0][1]} ' for xy in zip(combined.loc[combined['type']=='OrientationClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='OrientationClamp', ['fiber_plot_angle','depth']].shape[0]))])+''.join([f' L{xy[0][0]-10},{xy[0][1]} Z' if xy[1]==0 else f' L{xy[0][0]-10},{xy[0][1]}' for xy in zip(combined.loc[combined['type']=='OrientationClamp', ['fiber_plot_angle','depth']].values,range(combined.loc[combined['type']=='OrientationClamp', ['fiber_plot_angle','depth']].shape[0]))][::-1])

fig.update_layout(shapes=[dict(type="path", path=gozone_svg, fillcolor='rgba(154,205,50,0.2)', line=dict(width=0), layer='below'), dict(type="path", path=nogozone_svg, fillcolor='rgba(255,69,0,0.2)', line=dict(width=0), layer='below')])

# Add traces
fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='OrientationClamp', 'plot_angle'], y=combined.loc[combined['type']=='OrientationClamp', 'depth'], mode='markers', name='OrientationClamp', marker_color='forestgreen', customdata=combined.loc[combined['type']=='OrientationClamp', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='CentralizerClamp', 'plot_angle'], y=combined.loc[combined['type']=='CentralizerClamp', 'depth'], mode='markers', name='CentralizerClamp', marker_color='darkolivegreen', customdata=combined.loc[combined['type']=='CentralizerClamp', ['hadware_name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='Other', 'plot_angle'], y=combined.loc[combined['type']=='Other', 'depth'], mode='markers', name='Other', marker_color='green', customdata=combined.loc[combined['type']=='Other', ['hadware_name', 'angle_rounded']]))

# fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='PERFS', 'plot_angle'], y=combined.loc[combined['type']=='PERFS', 'depth'], mode='markers', name='PERFS', marker_color='navy', customdata=combined.loc[combined['type']=='PERFS', ['name', 'angle_rounded']]))

fig.add_trace(go.Scatter(x=combined.loc[combined['type']=='OrientationClamp', 'fiber_plot_angle'], y=combined.loc[combined['type']=='OrientationClamp', 'depth'], mode='lines+markers', name='Fiber Wire', marker_color='crimson', customdata=combined.loc[combined['type']=='OrientationClamp', ['hadware_name', 'fiber_angle_rounded']]))

fig.update_traces(hovertemplate='%{customdata[0]}<br>%{customdata[1]}')
fig.update_xaxes(range=[-185, 185])
fig.update_yaxes(autorange="reversed")
fig.update_layout(hovermode="y", title="Fiber cable orientation", legend_title="Type", legend_orientation="h", template="plotly", yaxis_title="Depth", xaxis_title='AngleFromHighSideClockwiseDegrees')
fig.show()

In [ ]:
features['type'].value_counts()
CC, OC, C= features['type'].value_counts()

pie = make_subplots(rows=1, cols=1, specs=[[{"type": "domain"}]])

pie.add_trace(go.Pie(
     values=[OC, CC, C],
     labels=['OrientationClamp', 'CentralizerClamp', 'Other'],
     domain=dict(x=[0, 0.5]),
     name="Blast Protector",
     hoverinfo='label+value',
     sort=False,
     marker_colors = ['forestgreen', 'darkolivegreen']
     ), 
     row=1, col=1)

pie.update_layout(title_text="Jewelry type", template="plotly")
pie.show()

In [ ]:
pio.write_html(fig, file='plotly/430_plot.html', auto_open=False)
pio.write_html(pie, file='plotly/430_pie.html', auto_open=False)